In [38]:
import astropy
import matplotlib
import numpy as np
import pandas as pd
import swsnet

from astropy.coordinates import SkyCoord
from concurrent.futures import ProcessPoolExecutor

In [34]:
def test_one_sws(in_arr):

    sra, sdec, cra, cdec, sindex = in_arr
    matches = []
    
    sref = SkyCoord(sra[sindex], sdec[sindex], frame='icrs', unit='deg')
    for index, value in enumerate(cra):
        cref = SkyCoord(cra[index], cdec[index], frame='icrs', unit='deg')
        sep = cref.separation(sref)
        if sep.arcminute <= 100:
            print(cass.aorkey.values[index], sws.tdt.values[0], sep.arcminute)
            matches.append((cass.aorkey.values[index], sws.tdt.values[0], sep.arcminute))
            
    return matches

In [21]:
cass = pd.read_pickle('cassis/metadata.pkl')
sws = pd.read_pickle('isosws_atlas/metadata.pkl')

In [24]:
cra = cass.ra.values
cdec = cass.dec.values

sra = sws.ra.values
sdec = sws.dec.values

In [35]:
myvar = []
for x in range(5):
    myvar.append((sra, sdec, cra, cdec, x))

In [41]:
pool = ProcessPoolExecutor(max_workers=4)
results = list(pool.map(test_one_sws, myvar))

22070016 37802225 93.5028247454558


In [42]:
results

[[], [], [], [], [('22070016', '37802225', 93.5028247454558)]]

In [40]:
sws;